In [20]:
import pymongo
from pymongo import MongoClient
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from sklearn import preprocessing
import warnings
import sys
sys.path.append('..')
from data_code import data
from nbi import *

ModuleNotFoundError: No module named 'maps'

## Import data

In [7]:
survey_records = pd.read_csv("/Users/AkshayKale/Documents/github/thesis/1992-2017-timeseries.csv")
survey_records.head()

/Users/AkshayKale/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Kind Of Material,Structure Type,Type of Wearing Surface,averageDailyTraffic,avgDailyTruckTraffic,countyCode,deck,designLoad,designatedInspectionFrequency,highwayAgencyDistrict,...,superstructure,year,yearBuilt,Age,State,Material,Structure type,Type of wearing surface,Average daily traffic,Average daily truck traffic
0,4,2,0,7760,10,3,4,4,24,1,...,7,1992,1947,Age: >=40 & <60,State: MA,Material: Steel Continuous,Structure Type: Stringer/Multi-beam or Girder,Type of WS: None (no additional concrete thick...,ADT: Heavy,ADTT: Heavy
1,3,10,0,19985,3,3,7,4,24,1,...,6,1992,1931,Age: >=60,State: MA,Material: Steel,Structure Type: Truss - Thru,Type of WS: None (no additional concrete thick...,ADT: Heavy,ADTT: Heavy
2,1,1,0,17242,10,3,7,4,24,1,...,7,1992,1953,Age: >=20 & <40,State: MA,Material: Concrete,Structure Type: Slab,Type of WS: None (no additional concrete thick...,ADT: Heavy,ADTT: Heavy
3,4,2,0,5860,4,3,7,4,24,1,...,7,1992,1959,Age: >=20 & <40,State: MA,Material: Steel Continuous,Structure Type: Stringer/Multi-beam or Girder,Type of WS: None (no additional concrete thick...,ADT: Heavy,ADTT: Moderate
4,3,3,0,5860,4,3,7,4,24,1,...,7,1992,1960,Age: >=20 & <40,State: MA,Material: Steel,Structure Type: Girder and Floorbeam System,Type of WS: None (no additional concrete thick...,ADT: Heavy,ADTT: Moderate


In [8]:
survey_records['structureNumber'].unique().shape

(771891,)

In [9]:
def createMapsLatest(survey_records, columns, key):
    """
    Returns dictionary of key : lastest value of the column
    for example:
                structure number (key): lastest value of the deck (value)
    Args:
        key (string):
        survey_records (pandas dataframe):
    
    Returns:
        all_dict (dict):
    
    """
    # Initialize list
    all_dict = []
    for col in columns:
        df = survey_records[[key, col]]
        struct_dict = {i:0 for i in df[key]}
       
        for k, v in zip(df[key], df[col]):
                struct_dict[k] = v
        all_dict.append(struct_dict)  
    
    return all_dict

def createMapsList(survey_records, columns, key):
    """
    Returns dictionary of key (string) : list value of the column (list)
    for example:
                structure number (key): list of value of the deck (value)
    Args:
        key (string):
        survey_records (pandas dataframe):
    
    Returns:
        all_dict (dict):
    
    """
    
    # Initialize list
    all_dict = []
    
    for col in columns:
        df = survey_records[[key, col]]
        struct_dict_list = {i:[] for i in df[key]}
        for k, v in zip(df[key], df[col]):
            struct_dict_list[k].append(v)
        all_dict.append(struct_dict_list)        
    
    return all_dict

# calling of the functions



def createGroupbyDf(key, df, list_of_maps):
    """
    Returns Groupby dataframe with values of the columns arranged according to their time series.
    
    Args:
        key (string): Key is the groupby criteria
        df (dataframe): the existing dataframe of lose individual records
        list_of_map (list): list of dictionary of columns in the df.
                            Each column is mapped to the key such tha'key' as the key,
                            and value as the 'value' of the column
    Returns:
        df_new (dataframe): a dataframe of columns grouped by key 'structure number'
    """
    #Select columns
    columns = list(df.columns)
    
    # initialize empty dataframe
    df_new = pd.DataFrame(columns = columns)
    
    # Setting column of structure number
    df_new[key] = df[key].unique()
    
    # mapping other column values to the structure number
    for number, col in enumerate(columns[1:]):
        df_new[col] = df_new[key].map(list_of_maps[number])
    
    return df_new   

## Creating columns for snowfall and freeze thaw

In [14]:
snowfall = pd.read_csv('/Users/AkshayKale/Google Drive/Data/scripts/climate.csv')

/Users/AkshayKale/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df_snowfall_freezethaw = snowfall[['structure_number', 'Year', 'NO_OF_FREEZE_THAW_CYCLES', 'no_of_snowfalls']]

In [16]:
df_snowfall_freezethaw.to_csv("/Users/AkshayKale/Google Drive/Data/scripts/climate_reduced.csv")

In [17]:
bridge_snowfall_map = dict(zip(df_snowfall_freezethaw['structure_number'], df_snowfall_freezethaw['no_of_snowfalls']))
bridge_freezethaw_map = dict(zip(df_snowfall_freezethaw['structure_number'], df_snowfall_freezethaw['NO_OF_FREEZE_THAW_CYCLES']))
bridge_year_climate_data_map = dict(zip(df_snowfall_freezethaw['structure_number'], df_snowfall_freezethaw['Year']))

In [18]:
bridge_climate_dict = createMapsLatest(df_snowfall_freezethaw, ['Year', 'NO_OF_FREEZE_THAW_CYCLES', 'no_of_snowfalls'], 'structure_number')

In [ ]:
bridge_climate_dict_of_list = createMapsList(df_snowfall_freezethaw, ['Year', 'NO_OF_FREEZE_THAW_CYCLES', 'no_of_snowfalls'], 'structure_number')

In [19]:
climate_columns = ['Year', 'NO_OF_FREEZE_THAW_CYCLE', 'no_of_snowfalls', 'Year list', 'NO_OF_FREEZE_THAW_CYCLE list', 'no_of_snowfalls list']